# GO-BGC Argopy Workshop

In [1]:
from argopy import DataFetcher  # This is the class to work with Argo data
from argopy import ArgoIndex  #  This is the class to work with Argo index
from argopy import ArgoNVSReferenceTables  # This is the class to retrieve data from Argo reference tables
from argopy import ArgoColors  # This is a class with usefull pre-defined colors
from argopy.plot import scatter_map, scatter_plot  # Functions to easily make maps and plots

# Make a fresh start
import argopy
argopy.set_options(cachedir='cache_bgc')
#argopy.reset_options()
argopy.clear_cache()

#
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import cmocean.cm as cmo
import xarray as xr
import cartopy.crs as ccrs
import pandas as pd
xr.set_options(display_expand_attrs = False)

# Load Storm Track

In [2]:
traj = pd.read_csv('sam_2021_ibtracs_data.csv')
traj_rows = traj.set_index(["ISO_TIME"])  # since this is a gridded data, it is useful to transorm from a table into an array 
traj_ds = xr.Dataset.from_dataframe(traj_rows)
traj_ds

<xarray.Dataset>
Dimensions:           (ISO_TIME: 117)
Coordinates:
  * ISO_TIME          (ISO_TIME) object '2021-09-22 18:00:00' ... '2021-10-07...
Data variables: (12/163)
    Unnamed: 0        (ISO_TIME) int64 0 1 2 3 4 5 6 ... 111 112 113 114 115 116
    SID               (ISO_TIME) object '2021266N10327' ... '2021266N10327'
    SEASON            (ISO_TIME) int64 2021 2021 2021 2021 ... 2021 2021 2021
    NUMBER            (ISO_TIME) int64 85 85 85 85 85 85 ... 85 85 85 85 85 85
    BASIN             (ISO_TIME) float64 nan nan nan nan nan ... nan nan nan nan
    SUBBASIN          (ISO_TIME) float64 nan nan nan nan nan ... nan nan nan nan
    ...                ...
    USA_SEARAD_NE     (ISO_TIME) object ' ' ' ' ' ' ' ' ' ' ... ' ' ' ' ' ' ' '
    USA_SEARAD_SE     (ISO_TIME) object ' ' ' ' ' ' ' ' ' ' ... ' ' ' ' ' ' ' '
    USA_SEARAD_SW     (ISO_TIME) object ' ' ' ' ' ' ' ' ' ' ... ' ' ' ' ' ' ' '
    USA_SEARAD_NW     (ISO_TIME) object ' ' ' ' ' ' ' ' ' ' ... ' ' ' ' ' ' ' '
    STORM_SPEED       (ISO_TIME) int64 12 13 14 16 16 16 ... 26 28 30 32 34 34
    STORM_DIR         (ISO_TIME) int64 280 279 278 277 279 ... 49 45 37 30 28

# Load Float Data

In [6]:
my_WMO = 5906436 #my hurricane ID
my_float= DataFetcher(ds='bgc', src='erddap',mode='expert', params='all',progress=True).float(my_WMO).load()

FSTimeoutError: 

In [ ]:
float_ds = my_float.data
float_ds = float_ds.argo.teos10(['CT','SA','SIG0'])

In [ ]:
float_ds = float_ds.argo.point2profile()
float_ds

In [ ]:
float_ds['PH_IN_SITU_TOTAL_ADJUSTED'] = float_ds['PH_IN_SITU_TOTAL_ADJUSTED'].where(float_ds['PH_IN_SITU_TOTAL_ADJUSTED'] != 99999.)
float_ds['NITRATE_ADJUSTED'] = float_ds['NITRATE_ADJUSTED'].where(float_ds['NITRATE_ADJUSTED'] != 99999.)
float_ds['CHLA_ADJUSTED'] = float_ds['CHLA_ADJUSTED'].where(float_ds['CHLA_ADJUSTED'] != 99999.)

In [ ]:
depth_min = 0
depth_max = 1000
step = 2
grid = np.arange(depth_min,depth_max,step)

In [ ]:
float_interp = float_ds.argo.interp_std_levels(grid)
float_interp

# Define Date of Intersection

In [ ]:
print('first storm traj: {}, last storm traj: {}'.format(traj_ds.ISO_TIME.min().data,traj_ds.ISO_TIME.max().data))

In [ ]:
print('first float prof: {}, last float prof: {}'.format(float_interp.TIME.min().data,float_interp.TIME.max().data))

In [ ]:
traj_ds['ISO_TIME'] = pd.DatetimeIndex(traj_ds['ISO_TIME'].values)
traj_ds

In [ ]:
float_interp.N_PROF

In [ ]:
N_PROF_during = int(float_interp.N_PROF.where(float_interp.TIME>=traj_ds.ISO_TIME.min()).where(float_interp.TIME<=traj_ds.ISO_TIME.max()).dropna('N_PROF').data)
N_PROF_during

In [ ]:
TIME_POSITION = []
for n in range(0,len(float_interp.N_PROF)):
    if n < N_PROF_during:
        TIME_POSITION.append('B')
        
    elif n== N_PROF_during:
        TIME_POSITION.append('D')
        
    elif n > N_PROF_during:
        TIME_POSITION.append('A')

In [ ]:
len(TIME_POSITION)

In [ ]:
float_interp['TIME_POSITION'] = TIME_POSITION
float_interp['TIME'] = float_interp.TIME.data
float_interp

# Plot Trajectories

In [ ]:
traj_ds

In [ ]:
float_interp

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
plt.scatter(float_interp.LONGITUDE,float_interp.LATITUDE,transform=ccrs.PlateCarree(),s=5,label='WMO: {}'.format(float_interp.PLATFORM_NUMBER[0].data))
plt.scatter(traj_ds.LON,traj_ds.LAT,transform=ccrs.PlateCarree(),s=5,label='Sam')
#ax.set_extent([-70,-50,20,30])
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.legend()
plt.title('Float & Sam Trajectories')

In [ ]:
import seaborn as sns

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
sns.scatterplot(x=float_interp.LONGITUDE,y=float_interp.LATITUDE,hue=float_interp.TIME_POSITION,transform=ccrs.PlateCarree(),s=5)
plt.scatter(traj_ds.LON,traj_ds.LAT,transform=ccrs.PlateCarree(),s=5,label='Sam')
#ax.set_extent([-70,-50,20,30])
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.legend()
plt.title('Float & Sam Trajectories')

In [ ]:
float_interp

In [ ]:
float_interp

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(421)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).CT.plot(y='PRES_INTERPOLATED',cmap=cmo.thermal)
plt.gca().invert_yaxis()
plt.title('Conservative Temperature')

plt.subplot(422)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).SA.plot(y='PRES_INTERPOLATED',cmap=cmo.haline)
plt.gca().invert_yaxis()
plt.title('Absolute Salinity')

plt.subplot(423)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).SIG0.plot(y='PRES_INTERPOLATED',cmap=cmo.dense)
plt.gca().invert_yaxis()
plt.title('Potential Density, 0dbar')

plt.subplot(424)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).DOXY_ADJUSTED.plot(y='PRES_INTERPOLATED',cmap=cmo.oxy)
plt.gca().invert_yaxis()
plt.title('Dissolved Oxygen')

plt.subplot(425)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).CHLA_ADJUSTED.plot(y='PRES_INTERPOLATED',cmap=cmo.algae)
plt.gca().invert_yaxis()
plt.title('Chlorophyll A')

plt.subplot(426)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).NITRATE_ADJUSTED.plot(y='PRES_INTERPOLATED',cmap=cmo.matter)
plt.gca().invert_yaxis()
plt.title('Nitrate')

plt.subplot(427)
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).BBP700.plot(y='PRES_INTERPOLATED',cmap=cmo.matter)
plt.gca().invert_yaxis()
plt.title('Optical Backscatter')

plt.subplots_adjust(hspace=0.35)

# Plot MLD Depth

In [ ]:
def get_MLD(ds,threshold=0.03,variable='SIG0',dim1='N_PROF',dim2='PRES_INTERPOLATED'):
    '''Takes an xarray and returns a new coordinate "MLD" or mixed layer depth for each profile, defined using the density threshold from the surface.
    ds: xarray with profile and pressure dimensions
    threshold: density value that defines the boundary of the mixed layer, default=0.03
    variable: density coordinate, default='SIG0'
    dim1: profile dimension, default='N_PROF'
    dim2: pressure dimension, default='PRES_INTERPOLATED'
    '''
    
    MLD_li = []
    
    for n in range(0, len(ds[dim1])):
        SIG0_surface = ds.isel({dim1:n})[variable].isel({dim2:0})
        SIG0_diff    = SIG0_surface + threshold
        MLD_ds       = SIG0_surface.where(ds.isel({dim1:n})[variable] < SIG0_diff)
        MLD          = MLD_ds.dropna(dim2).isel({dim2:-1})[dim2].values
        MLD_li.append(MLD)
        
    return ds.assign_coords(MLD=(dim1,MLD_li))

In [ ]:
float_interp = get_MLD(float_interp)

In [ ]:
float_interp.sel(PRES_INTERPOLATED=slice(0,250)).SIG0.plot(y='PRES_INTERPOLATED',cmap=cmo.dense)
plt.gca().invert_yaxis()
float_interp.MLD.plot(color='k')
plt.title('Densite with MLD Contour')